In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.metrics import confusion_matrix, classification_report
from model_training import *

In [4]:
metadata_file = "/media/romulo/6237-3231/urban_sound_challenge/spectrograms_128_array/location_mapping.csv"
file_base_location = "/media/romulo/6237-3231/urban_sound_challenge/spectrograms_128_array/"

In [ ]:
train_model(metadata_file,file_base_location,
                data_dim=(128,128),
                batch_size=128,
                n_classes=10,
                training_folds = [10,2,3,4,5,6,7,8],
                validation_folds = [9],
                shuffle=True)

W0718 23:30:25.919289 140685981546304 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 23:30:25.933998 140685981546304 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 23:30:25.936570 140685981546304 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 23:30:25.960018 140685981546304 deprecation_wrapper.py:119] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.

[ 2 10  4  7  6  3  8  5] [9]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 24)      624       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 58, 48)        28848     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 48)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 48)        57648     
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30000)    

W0718 23:30:26.223583 140685981546304 deprecation.py:323] From /home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
232/493 [=============>................] - ETA: 6:48 - loss: 15.4103 - acc: 0.1248

Process ForkPoolWorker-1:
Process ForkPoolWorker-12:
Process ForkPoolWorker-4:
Process ForkPoolWorker-11:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-9:
Process ForkPoolWorker-5:
Process ForkPoolWorker-10:
Process ForkPoolWorker-2:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most

Epoch 1/1


  File "/home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/numpy/lib/format.py", line 567, in _read_array_header
    dtype = descr_to_dtype(d['descr'])
KeyboardInterrupt
  File "/home/romulo/github/pyvenv/urbansounds8k/lib/python3.6/site-packages/numpy/lib/format.py", line 262, in descr_to_dtype
    def descr_to_dtype(descr):
KeyboardInterrupt
Process ForkPoolWorker-17:
Process ForkPoolWorker-18:
Process ForkPoolWorker-19:
Process ForkPoolWorker-14:
Process ForkPoolWorker-24:
Process ForkPoolWorker-20:
Process ForkPoolWorker-16:
Process ForkPoolWorker-22:
Process ForkPoolWorker-13:
Process ForkPoolWorker-21:
Process ForkPoolWorker-15:
Process ForkPoolWorker-23:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last